In [1]:
import tensorflow as tf
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.boston_housing.load_data()

In [2]:
X_train.shape

(404, 13)

In [3]:
from tensorflow import keras
def build_model(n_hidden, n_neurons, optimizer, learning_rate, momentum=0):
    model = tf.keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=X_train.shape[1:]))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(1))
    if optimizer == "sgd":
        optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
    elif optimizer == "nesterov":
        optimizer = keras.optimizers.SGD(learning_rate=learning_rate, nesterov=True)
    if optimizer == "momentum":
        optimizer = keras.optimizers.SGD(learning_rate=learning_rate, momentum=momentum)
    if optimizer == "adam":
        optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss="mse", metrics=["mae"], optimizer=optimizer)
    return model

In [4]:
import os
import time
root_logdir = os.path.join(os.curdir, "tb_logs")

def get_dir(name, value):
    ts = int(time.time())
    run_id = f"{ts}_{name}_{value}"
    return os.path.join(root_logdir, run_id)

In [5]:
es_cb = tf.keras.callbacks.EarlyStopping(patience=10, min_delta=1.00)

In [6]:
import numpy as np
tf.keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

First experiment

In [7]:
lr = [10**(-6), 10**(-5), 10**(-4)]
name = "lr"
results_lr = []
for value in lr:
    tensorboard_cb = tf.keras.callbacks.TensorBoard(get_dir(name, value))
    model = build_model(1, 25, "sgd", value)
    model.fit(X_train, y_train, epochs=100, validation_split=0.1, callbacks=[tensorboard_cb, es_cb])
    score = model.evaluate(X_test, y_test)
    results_lr.append((value, score[0], score[1]))

Epoch 1/100
12/12 [==============================] - 1s 19ms/step - loss: 3401.3970 - mae: 33.4821 - val_loss: 311.0516 - val_mae: 14.4932
Epoch 2/100
12/12 [==============================] - 0s 6ms/step - loss: 254.7935 - mae: 12.1135 - val_loss: 137.1021 - val_mae: 9.5869
Epoch 3/100
12/12 [==============================] - 0s 8ms/step - loss: 206.4626 - mae: 11.0221 - val_loss: 111.4679 - val_mae: 8.4720
Epoch 4/100
12/12 [==============================] - 0s 7ms/step - loss: 190.3381 - mae: 10.6453 - val_loss: 103.1058 - val_mae: 8.1271
Epoch 5/100
12/12 [==============================] - 0s 10ms/step - loss: 181.7432 - mae: 10.3951 - val_loss: 92.0520 - val_mae: 7.5028
Epoch 6/100
12/12 [==============================] - 0s 7ms/step - loss: 167.9646 - mae: 10.0638 - val_loss: 91.0381 - val_mae: 7.4938
Epoch 7/100
12/12 [==============================] - 0s 7ms/step - loss: 160.2945 - mae: 9.7564 - val_loss: 86.1156 - val_mae: 7.1749
Epoch 8/100
12/12 [=============================

In [8]:
results_lr

[(1e-06, 75.78328704833984, 6.495291709899902),
 (1e-05, 66.48982238769531, 5.946558952331543),
 (0.0001, 187.84597778320312, 11.154184341430664)]

In [9]:
import pickle
with open('lr.pkl', 'wb') as fp:
    pickle.dump(results_lr, fp)

In [10]:
%load_ext tensorboard
%tensorboard --logdir ./tb_logs

Reusing TensorBoard on port 6006 (pid 6480), started 4:31:37 ago. (Use '!kill 6480' to kill it.)

In [11]:
hl = list(range(4))
name = "hl"
results_hl = []
for value in hl:
    tensorboard_cb = tf.keras.callbacks.TensorBoard(get_dir(name, value))
    model = build_model(value, 25, "sgd", 10**(-5))
    model.fit(X_train, y_train, epochs=100, validation_split=0.1, callbacks=[tensorboard_cb, es_cb])
    score = model.evaluate(X_test, y_test)
    results_hl.append((value, score[0], score[1]))

Epoch 1/100
12/12 [==============================] - 1s 17ms/step - loss: 21292295559556628480.0000 - mae: 1202941952.0000 - val_loss: 21449109297692739108864.0000 - val_mae: 143597600768.0000
Epoch 2/100
12/12 [==============================] - 0s 12ms/step - loss: inf - mae: 539821520172613632.0000 - val_loss: inf - val_mae: 67373058777090621440.0000
Epoch 3/100
12/12 [==============================] - 0s 8ms/step - loss: inf - mae: 276100940946972912849518592.0000 - val_loss: inf - val_mae: 26818146889840090426529284096.0000
Epoch 4/100
12/12 [==============================] - 0s 6ms/step - loss: inf - mae: 109274026944394145150861008902815744.0000 - val_loss: inf - val_mae: inf
Epoch 5/100
12/12 [==============================] - 0s 9ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 6/100
12/12 [==============================] - 0s 19ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 7/100
12/12 [==============================] - 0s 9ms/step - los

In [12]:
with open('hl.pkl', 'wb') as fp:
    pickle.dump(results_hl, fp)
    
results_hl

[(0, nan, nan),
 (1, 590.48486328125, 22.52201271057129),
 (2, 59.99591064453125, 5.643824100494385),
 (3, 63.30836486816406, 5.930777072906494)]

In [13]:
%tensorboard --logdir ./tb_logs

Reusing TensorBoard on port 6006 (pid 6480), started 4:31:53 ago. (Use '!kill 6480' to kill it.)

In [14]:
nn = [5, 25, 125]
name = "nn"
results_nn = []
for value in nn:
    tensorboard_cb = tf.keras.callbacks.TensorBoard(get_dir(name, value))
    model = build_model(1, value, "sgd", 10**(-5))
    model.fit(X_train, y_train, epochs=100, validation_split=0.1, callbacks=[tensorboard_cb, es_cb])
    score = model.evaluate(X_test, y_test)
    results_nn.append((value, score[0], score[1]))

Epoch 1/100
12/12 [==============================] - 1s 23ms/step - loss: 14378.0225 - mae: 54.8116 - val_loss: 505.2232 - val_mae: 21.5068
Epoch 2/100
12/12 [==============================] - 0s 7ms/step - loss: 594.5654 - mae: 22.4790 - val_loss: 504.9935 - val_mae: 21.5015
Epoch 3/100
12/12 [==============================] - 0s 12ms/step - loss: 594.3253 - mae: 22.4736 - val_loss: 504.7639 - val_mae: 21.4961
Epoch 4/100
12/12 [==============================] - 0s 14ms/step - loss: 594.0853 - mae: 22.4682 - val_loss: 504.5320 - val_mae: 21.4908
Epoch 5/100
12/12 [==============================] - 0s 8ms/step - loss: 593.8430 - mae: 22.4629 - val_loss: 504.2996 - val_mae: 21.4853
Epoch 6/100
12/12 [==============================] - 0s 8ms/step - loss: 593.6001 - mae: 22.4574 - val_loss: 504.0693 - val_mae: 21.4800
Epoch 7/100
12/12 [==============================] - 0s 16ms/step - loss: 593.3593 - mae: 22.4521 - val_loss: 503.8376 - val_mae: 21.4746
Epoch 8/100
12/12 [================

In [15]:
with open('nn.pkl', 'wb') as fp:
    pickle.dump(results_nn, fp)
    
results_nn

[(5, 591.2012329101562, 22.53790855407715),
 (25, 70.50782775878906, 6.389472484588623),
 (125, 62.27962875366211, 5.80792236328125)]

In [16]:
%tensorboard --logdir ./tb_logs

Reusing TensorBoard on port 6006 (pid 6480), started 4:32:10 ago. (Use '!kill 6480' to kill it.)

In [17]:
opt = ["sgd", "nesterov", "momentum", "adam"]
name = "opt"
results_opt= []
for value in opt:
    tensorboard_cb = tf.keras.callbacks.TensorBoard(get_dir(name, value))
    model = build_model(1, 25, value, 10**(-5), 0.5)
    model.fit(X_train, y_train, epochs=100, validation_split=0.1, callbacks=[tensorboard_cb, es_cb])
    score = model.evaluate(X_test, y_test)
    results_opt.append((value, score[0], score[1]))

Epoch 1/100
12/12 [==============================] - 1s 25ms/step - loss: 13018.1240 - mae: 65.2370 - val_loss: 529.5540 - val_mae: 22.2617
Epoch 2/100
12/12 [==============================] - 0s 9ms/step - loss: 487.2162 - mae: 19.8275 - val_loss: 99.9038 - val_mae: 7.6155
Epoch 3/100
12/12 [==============================] - 0s 9ms/step - loss: 156.7772 - mae: 9.7153 - val_loss: 71.1782 - val_mae: 6.6300
Epoch 4/100
12/12 [==============================] - 0s 23ms/step - loss: 110.3769 - mae: 8.0015 - val_loss: 51.0345 - val_mae: 5.5003
Epoch 5/100
12/12 [==============================] - 0s 9ms/step - loss: 91.7442 - mae: 7.1407 - val_loss: 45.3149 - val_mae: 5.1759
Epoch 6/100
12/12 [==============================] - 0s 11ms/step - loss: 84.6806 - mae: 6.8575 - val_loss: 43.8818 - val_mae: 5.1191
Epoch 7/100
12/12 [==============================] - 0s 14ms/step - loss: 84.0727 - mae: 6.7035 - val_loss: 40.4575 - val_mae: 4.9183
Epoch 8/100
12/12 [==============================] - 0s

In [18]:
with open('opt.pkl', 'wb') as fp:
    pickle.dump(results_opt, fp)
    
results_opt

[('sgd', 70.92853546142578, 6.284383296966553),
 ('nesterov', 80.82572937011719, 6.687215805053711),
 ('momentum', 100.24166107177734, 8.472890853881836),
 ('adam', 3027.010498046875, 46.63709259033203)]

In [19]:
%tensorboard --logdir ./tb_logs

Reusing TensorBoard on port 6006 (pid 6480), started 4:32:35 ago. (Use '!kill 6480' to kill it.)

In [20]:
mom = [0.1, 0.5, 0.9]
name = "mom"
results_mom = []
for value in mom:
    tensorboard_cb = tf.keras.callbacks.TensorBoard(get_dir(name, value))
    model = build_model(1, 25, "momentum", 10**(-5), value)
    model.fit(X_train, y_train, epochs=100, validation_split=0.1, callbacks=[tensorboard_cb, es_cb])
    score = model.evaluate(X_test, y_test)
    results_mom.append((value, score[0], score[1]))

Epoch 1/100
12/12 [==============================] - 1s 26ms/step - loss: 12535.2217 - mae: 63.7885 - val_loss: 497.5861 - val_mae: 21.2019
Epoch 2/100
12/12 [==============================] - 0s 10ms/step - loss: 588.1895 - mae: 22.0781 - val_loss: 495.5449 - val_mae: 21.0787
Epoch 3/100
12/12 [==============================] - 0s 13ms/step - loss: 586.9178 - mae: 21.9877 - val_loss: 494.1761 - val_mae: 20.9802
Epoch 4/100
12/12 [==============================] - 0s 15ms/step - loss: 586.0182 - mae: 21.9478 - val_loss: 492.1589 - val_mae: 20.8182
Epoch 5/100
12/12 [==============================] - 0s 8ms/step - loss: 584.9926 - mae: 21.9250 - val_loss: 490.8977 - val_mae: 20.6986
Epoch 6/100
12/12 [==============================] - 0s 8ms/step - loss: 584.4433 - mae: 21.9150 - val_loss: 490.0975 - val_mae: 20.6194
Epoch 7/100
12/12 [==============================] - 0s 8ms/step - loss: 583.9919 - mae: 21.9117 - val_loss: 489.3862 - val_mae: 20.5756
Epoch 8/100
12/12 [================

In [21]:
with open('mom.pkl', 'wb') as fp:
    pickle.dump(results_mom, fp)
    
results_mom

[(0.1, 78.12017822265625, 6.560732364654541),
 (0.5, 88.1409912109375, 6.952456474304199),
 (0.9, 414.5842590332031, 18.202756881713867)]

In [22]:
%tensorboard --logdir ./tb_logs

Reusing TensorBoard on port 6006 (pid 6480), started 4:33:02 ago. (Use '!kill 6480' to kill it.)

In [23]:
param_distribs = {
    "model__n_hidden": [0, 1, 2, 3],
    "model__n_neurons": [5, 25, 125],
    "model__learning_rate": [10**-6, 10**-5, 10**-4],
    "model__optimizer": ["sgd", "nesterov", "momentum", "adam"],
    "model__momentum": [0.1, 0.5, 0.9]
}

In [24]:
import scikeras
from scikeras.wrappers import KerasRegressor

es = tf.keras.callbacks.EarlyStopping(patience=10, min_delta=1.0, verbose=1)

keras_reg = KerasRegressor(build_model, callbacks=[es])

In [25]:
from sklearn.model_selection import RandomizedSearchCV

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=20, cv=3, verbose=2)
rnd_search_cv.fit(X_train, y_train, epochs=100, validation_split=0.1)

Fitting 3 folds for each of 20 candidates, totalling 60 fits
Epoch 1/100
8/8 [==============================] - 1s 32ms/step - loss: 20131.1855 - mae: 72.9395 - val_loss: 445.7802 - val_mae: 20.1908
Epoch 2/100
8/8 [==============================] - 0s 12ms/step - loss: 592.9150 - mae: 22.5755 - val_loss: 441.5637 - val_mae: 20.0861
Epoch 3/100
8/8 [==============================] - 0s 13ms/step - loss: 588.0016 - mae: 22.4655 - val_loss: 437.0992 - val_mae: 19.9746
Epoch 4/100
8/8 [==============================] - 0s 10ms/step - loss: 583.0701 - mae: 22.3559 - val_loss: 432.8120 - val_mae: 19.8670
Epoch 5/100
8/8 [==============================] - 0s 13ms/step - loss: 578.1187 - mae: 22.2449 - val_loss: 428.2100 - val_mae: 19.7509
Epoch 6/100
8/8 [==============================] - 0s 9ms/step - loss: 572.9350 - mae: 22.1268 - val_loss: 423.8127 - val_mae: 19.6392
Epoch 7/100
8/8 [==============================] - 0s 21ms/step - loss: 567.9205 - mae: 22.0140 - val_loss: 419.0535 - val

/Users/hania/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/hania/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/hania/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 418, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/Users/hania/opt/anaconda3/lib/python3.8/site-packages/scikeras/wrappers.py", line 1100, in score
    return self.scorer(y, y_pred, sample_weight=sample_weight, **score_args)
  File "/Users/hania/opt/anaconda3/lib/python3.8/site-packages/scikeras/wrappers.py", line 1697, in scorer
    return sklearn_r2_score(y_true, y_pred, **kwargs)
  File "/Users/hania/opt/anaconda3/lib/python3.8/site-packages/

8/8 [==============================] - 1s 30ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 2/100
8/8 [==============================] - 0s 12ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 3/100
8/8 [==============================] - 0s 8ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 4/100
8/8 [==============================] - 0s 13ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 5/100
8/8 [==============================] - 0s 10ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 6/100
8/8 [==============================] - 0s 9ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 7/100
8/8 [==============================] - 0s 9ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 8/100
8/8 [==============================] - 0s 12ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 9/100
8/8 [==============================] - 0s 11ms/step - loss:

/Users/hania/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/hania/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/hania/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 418, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/Users/hania/opt/anaconda3/lib/python3.8/site-packages/scikeras/wrappers.py", line 1100, in score
    return self.scorer(y, y_pred, sample_weight=sample_weight, **score_args)
  File "/Users/hania/opt/anaconda3/lib/python3.8/site-packages/scikeras/wrappers.py", line 1697, in scorer
    return sklearn_r2_score(y_true, y_pred, **kwargs)
  File "/Users/hania/opt/anaconda3/lib/python3.8/site-packages/

8/8 [==============================] - 1s 25ms/step - loss: 1774.2852 - mae: 40.6237 - val_loss: 1679.4150 - val_mae: 40.2672
Epoch 2/100
8/8 [==============================] - 0s 12ms/step - loss: 1774.0135 - mae: 40.6205 - val_loss: 1679.1373 - val_mae: 40.2639
Epoch 3/100
8/8 [==============================] - 0s 14ms/step - loss: 1773.7402 - mae: 40.6172 - val_loss: 1678.8593 - val_mae: 40.2606
Epoch 4/100
8/8 [==============================] - 0s 12ms/step - loss: 1773.4664 - mae: 40.6140 - val_loss: 1678.5830 - val_mae: 40.2573
Epoch 5/100
8/8 [==============================] - 0s 14ms/step - loss: 1773.1968 - mae: 40.6109 - val_loss: 1678.3060 - val_mae: 40.2540
Epoch 6/100
8/8 [==============================] - 0s 12ms/step - loss: 1772.9227 - mae: 40.6076 - val_loss: 1678.0302 - val_mae: 40.2507
Epoch 7/100
8/8 [==============================] - 0s 9ms/step - loss: 1772.6511 - mae: 40.6045 - val_loss: 1677.7522 - val_mae: 40.2474
Epoch 8/100
8/8 [==============================

...

/Users/hania/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/hania/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/hania/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 418, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/Users/hania/opt/anaconda3/lib/python3.8/site-packages/scikeras/wrappers.py", line 1100, in score
    return self.scorer(y, y_pred, sample_weight=sample_weight, **score_args)
  File "/Users/hania/opt/anaconda3/lib/python3.8/site-packages/scikeras/wrappers.py", line 1697, in scorer
    return sklearn_r2_score(y_true, y_pred, **kwargs)
  File "/Users/hania/opt/anaconda3/lib/python3.8/site-packages/

8/8 [==============================] - 1s 24ms/step - loss: 1576.4769 - mae: 31.0177 - val_loss: 108.1109 - val_mae: 8.9697
Epoch 2/100
8/8 [==============================] - 0s 10ms/step - loss: 98.0885 - mae: 7.9139 - val_loss: 54.9634 - val_mae: 5.7934
Epoch 3/100
8/8 [==============================] - 0s 13ms/step - loss: 85.2625 - mae: 6.8335 - val_loss: 52.0326 - val_mae: 5.6580
Epoch 4/100
8/8 [==============================] - 0s 9ms/step - loss: 84.3973 - mae: 6.7652 - val_loss: 50.1613 - val_mae: 5.5616
Epoch 5/100
8/8 [==============================] - 0s 8ms/step - loss: 84.2943 - mae: 6.7329 - val_loss: 49.0771 - val_mae: 5.5018
Epoch 6/100
8/8 [==============================] - 0s 9ms/step - loss: 84.0199 - mae: 6.7467 - val_loss: 47.2891 - val_mae: 5.4162
Epoch 7/100
8/8 [==============================] - 0s 9ms/step - loss: 83.3032 - mae: 6.6852 - val_loss: 45.7361 - val_mae: 5.3428
Epoch 8/100
8/8 [==============================] - 0s 10ms/step - loss: 83.4284 - mae: 6

/Users/hania/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:969: UserWarning: One or more of the test scores are non-finite: [            nan -1.92049806e+01  2.14922144e-01 -2.39302831e-01
             nan  8.21757097e-02             nan             nan
 -3.65483113e+02             nan             nan  3.13113283e-01
 -3.31349732e+02             nan -6.47096265e+00  1.42239177e-01
             nan  2.75028300e-03 -5.40451461e-01  2.14098850e-01]
  warnings.warn(


12/12 [==============================] - 1s 17ms/step - loss: 780.3152 - mae: 17.8700 - val_loss: 52.5324 - val_mae: 6.0175
Epoch 2/100
12/12 [==============================] - 0s 6ms/step - loss: 87.6351 - mae: 6.8682 - val_loss: 43.6857 - val_mae: 5.2989
Epoch 3/100
12/12 [==============================] - 0s 6ms/step - loss: 84.1127 - mae: 6.6473 - val_loss: 35.6155 - val_mae: 4.4513
Epoch 4/100
12/12 [==============================] - 0s 7ms/step - loss: 79.1199 - mae: 6.3947 - val_loss: 33.1547 - val_mae: 4.1252
Epoch 5/100
12/12 [==============================] - 0s 8ms/step - loss: 76.9004 - mae: 6.2778 - val_loss: 31.7412 - val_mae: 4.2325
Epoch 6/100
12/12 [==============================] - 0s 8ms/step - loss: 74.3124 - mae: 6.2095 - val_loss: 34.4787 - val_mae: 4.5321
Epoch 7/100
12/12 [==============================] - 0s 7ms/step - loss: 72.7393 - mae: 5.9828 - val_loss: 28.9629 - val_mae: 3.8538
Epoch 8/100
12/12 [==============================] - 0s 6ms/step - loss: 72.25

RandomizedSearchCV(cv=3,
                   estimator=KerasRegressor(callbacks=[<keras.callbacks.EarlyStopping object at 0x7fb97d108c40>], model=<function build_model at 0x7fb9987dd310>),
                   n_iter=20,
                   param_distributions={'model__learning_rate': [1e-06, 1e-05,
                                                                 0.0001],
                                        'model__momentum': [0.1, 0.5, 0.9],
                                        'model__n_hidden': [0, 1, 2, 3],
                                        'model__n_neurons': [5, 25, 125],
                                        'model__optimizer': ['sgd', 'nesterov',
                                                             'momentum',
                                                             'adam']},
                   verbose=2)

In [26]:
rnd_search_cv.best_params_

{'model__optimizer': 'momentum',
 'model__n_neurons': 125,
 'model__n_hidden': 1,
 'model__momentum': 0.5,
 'model__learning_rate': 1e-06}

In [27]:
with open('rnd_search.pkl', 'wb') as fp:
    pickle.dump(rnd_search_cv.best_params_, fp)